In [165]:
# Hack to allow imports from directory above. Once the package is fully 
# functional, remove this and instead just install and import the package
import sys
sys.path.append("..")

import pandas as pd

from datetime import datetime
from portfolio_manager.portfolio import InvestmentPortfolio, load_portfolio
from portfolio_manager.return_calculators import (ReturnCalculator,
                                                  SimpleReturnCalculator, 
                                                  TimeWeightedReturnCalculator)

### Create a portfolio and perform some transactions

In [166]:
# Create a portfolio
portfolio = InvestmentPortfolio(name='T212')

In [167]:
# Make some deposits
portfolio.deposit(10)
portfolio.deposit(20, portfolio_value_before_deposit=15)

In [168]:
# Update the current portfolio value
portfolio.update_portfolio_value(75)

In [169]:
# Withdraw some funds
portfolio.withdraw(12.50, date=datetime(2022, 1, 1))

In [170]:
# View the portfolio history
portfolio.portfolio_history

[{'date': datetime.datetime(2021, 1, 23, 12, 50, 7, 829173),
  'total_deposited': 10,
  'current_portfolio_value': 10,
  'transaction_type': 'deposit'},
 {'date': datetime.datetime(2021, 1, 23, 12, 50, 7, 829272),
  'total_deposited': 10,
  'current_portfolio_value': 15,
  'transaction_type': 'update_portfolio_value'},
 {'date': datetime.datetime(2021, 1, 23, 12, 50, 7, 829272),
  'total_deposited': 30,
  'current_portfolio_value': 35,
  'transaction_type': 'deposit'},
 {'date': datetime.datetime(2021, 1, 23, 12, 50, 7, 915025),
  'total_deposited': 30,
  'current_portfolio_value': 75,
  'transaction_type': 'update_portfolio_value'},
 {'date': datetime.datetime(2022, 1, 1, 0, 0),
  'total_deposited': 17.5,
  'current_portfolio_value': 62.5,
  'transaction_type': 'withdrawal'}]

In [171]:
# View the portfolio history as a dataframe
df = pd.DataFrame(portfolio.portfolio_history, columns=['date', 'total_deposited', 'current_portfolio_value', 'transaction_type'])
df

,date,total_deposited,current_portfolio_value,transaction_type
0,2021-01-23 12:50:07.829173,10.0,10.0,deposit
1,2021-01-23 12:50:07.829272,10.0,15.0,update_portfolio_value
2,2021-01-23 12:50:07.829272,30.0,35.0,deposit
3,2021-01-23 12:50:07.915025,30.0,75.0,update_portfolio_value
4,2022-01-01 00:00:00.000000,17.5,62.5,withdrawal


### Standard return calculator

In [172]:
simple_return_calculator = SimpleReturnCalculator()
portfolio_return = simple_return_calculator.calculate_return(portfolio, annualised=False)

In [173]:
portfolio_return = simple_return_calculator.calculate_return(portfolio, annualised=False)
portfolio_return

257.14

In [174]:
portfolio_return = simple_return_calculator.calculate_return(portfolio, annualised=True)
portfolio_return

287.37

### Time-weighted return calculator

In [175]:
time_weighted_return_calculator = TimeWeightedReturnCalculator()
portfolio_return = time_weighted_return_calculator.calculate_return(portfolio, annualised=False)
portfolio_return

221.43

In [176]:
time_weighted_return_calculator = TimeWeightedReturnCalculator()
annualised_portfolio_return = time_weighted_return_calculator.calculate_return(portfolio, annualised=True)
annualised_portfolio_return

246.3

### Saving/loading a portfolio

In [177]:
portfolio.save_portfolio()

In [178]:
p = load_portfolio('T212')

In [179]:
p.total_deposited

17.5

In [180]:
p.current_portfolio_value

62.5

In [181]:
p.name

'T212'

In [137]:
df

,date,total_deposited,current_portfolio_value,transaction_type
0,2021-01-23 12:12:29.230094,10.0,10.0,deposit
1,2021-01-23 12:12:29.230152,10.0,15.0,update_portfolio_value
2,2021-01-23 12:12:29.230152,30.0,35.0,deposit
3,2021-01-23 12:12:29.306021,30.0,75.0,update_portfolio_value
4,2022-01-01 00:00:00.000000,17.5,62.5,withdrawal


In [138]:
dandw = df.loc[df['transaction_type'].isin(['deposit','withdrawal'])]
dandw

,date,total_deposited,current_portfolio_value,transaction_type
0,2021-01-23 12:12:29.230094,10.0,10.0,deposit
2,2021-01-23 12:12:29.230152,30.0,35.0,deposit
4,2022-01-01 00:00:00.000000,17.5,62.5,withdrawal


In [156]:
total_deposited = df.loc[:, df.columns.intersection(['total_deposited'])].diff()
total_deposited

,total_deposited
0,NaN
2,20.0
4,-12.5


In [140]:
diff_df = total_deposited.diff()
diff_df

,total_deposited
0,NaN
2,20.0
4,-12.5


In [142]:
diff_list = diff_df['total_deposited'].tolist()
diff_list


[nan, 20.0, -12.5]

In [192]:
import numpy
mwr_arr = []
mwr_arr.append(-df['total_deposited'][0])
mwr_arr.extend(numpy.negative(diff_list[1:]))
mwr_arr

[-10.0, -20.0, 12.5]

In [194]:
mwr_arr
mwr_arr.append(df['current_portfolio_value'].iloc[-1])
mwr_arr

[-10.0, -20.0, 12.5, 62.5]

In [195]:
numpy.irr(mwr_arr)

0.5214775427188729